## data

In [1]:
import pandas as pd
import re
from string import punctuation
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
# 加载原始数据
data = pd.read_csv('./AI_Human.csv')

# 数据采样与清洗
ai_samples = data[data['generated'] == 1]
human_samples = data[data['generated'] == 0]
data = pd.concat([ai_samples.sample(n=5000, random_state=42), human_samples.sample(n=5000, random_state=42)])
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

# 清洗函数
def remove_punc(text):
    return ''.join([char for char in text if char not in punctuation])

def remove_stop(text):
    stops = set(stopwords.words('english'))
    return " ".join([word for word in text.split() if word.lower() not in stops])

# 文本清洗
data['cleaned'] = data['text'].str.lower()
data['cleaned'] = data['cleaned'].apply(lambda x: re.sub(r'https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE))
data['cleaned'] = data['cleaned'].apply(lambda x: re.sub(r'<.*?>', '', x))
data['cleaned'] = data['cleaned'].apply(remove_punc)
#data['cleaned'] = data['cleaned'].apply(remove_stop)



# 转换为指令微调格式
formatted_data = []
for _, row in data.iterrows():
    formatted_data.append({
        "instruction": "Classify the text as either human-written or AI-generated. Respond with 'Human-written' or 'AI-generated'.",
        "input": row['cleaned'],
        "output": "AI-generated" if row['generated'] == 1 else "Human-written"
    })

# 查看处理后的前几条数据
for entry in formatted_data[:5]:
    print(entry)

{'instruction': "Classify the text as either human-written or AI-generated. Respond with 'Human-written' or 'AI-generated'.", 'input': 'dear principal\n\nwe have been hearing quite a lot about the subject of doing community service lately and my classmates have chosen me to write you this letter about what we think you should do and some reasons as to why you should as we all know community service is a lot of work bkt it is also worth the work because we get a better town and community\n\ncommunity service is a very important matter in which i believe everyone should be involved whether it is cleaning a local park or helping other students it is a very egregious activity for the community it would also attract good attention to the school and promote other schools to do the same\n\ncommunity service is utterly important not just to keep the town clean bkt also to promote good habits and motivate students to help their neighborhoods or wherever they get in life some ideas for services 

In [2]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

In [3]:
tokenizer = AutoTokenizer.from_pretrained('/root/autodl-tmp/ZhipuAI/glm-4-9b-chat', use_fast=False, trust_remote_code=True)
# 将tokenizer的pad_token设置为eos_token，这样在进行填充时，会使用eos_token作为填充符号
tokenizer.pad_token = tokenizer.eos_token


In [4]:
def process_func(example):
    MAX_LENGTH = 384  # 定义最大长度
    input_ids, attention_mask, labels = [], [], []

    # 定义指令和响应的格式，注意替换成你的实际需求
    instruction = tokenizer(
        (f"[gMASK]<sop><|system|>\nClassify the text as either human-written or AI-generated.\n<|user|>\n"
         f"{example['instruction']}\nInput: {example['input']}<|assistant|>\n").strip(),
        add_special_tokens=False
    )
    response = tokenizer(f"{example['output']} <|endoftext|>", add_special_tokens=False)

    # 拼接 input_ids 和 attention_mask
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]

    # 构建 labels，instruction 部分不计算损失，填充为 -100
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]

    # 截断至最大长度
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }


In [5]:
df = pd.DataFrame(formatted_data)

# 划分数据集
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)  # 10% 作为测试集

# 转换为 Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# 对训练集和测试集分别进行编码
tokenized_train_dataset = train_dataset.map(process_func, remove_columns=train_dataset.column_names)
tokenized_test_dataset = test_dataset.map(process_func, remove_columns=test_dataset.column_names)

print(f"训练集大小: {len(tokenized_train_dataset)}")
print(f"测试集大小: {len(tokenized_test_dataset)}")

tokenized_id=tokenized_train_dataset
tokenized_id

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

训练集大小: 9000
测试集大小: 1000


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 9000
})

In [6]:
print(tokenizer.decode(tokenized_id[0]['input_ids']))
print(tokenized_id[0]['input_ids'])
print(tokenizer.decode([151331, 151333, 151335]))
print(tokenizer.encode('[gMASK]<sop><|system|>', add_special_tokens=False))
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"])))


[gMASK] <sop> <|system|> 
Classify the text as either human-written or AI-generated.
 <|user|> 
Classify the text as either human-written or AI-generated. Respond with 'Human-written' or 'AI-generated'.
Input:  the advantages of limiting car usage

in recent years there has been a growing trend of people limiting their car usage this is due to a number of factors including the high cost of owning and operating a car the environmental impact of car emissions and the desire for a more active and healthy lifestyle

there are a number of advantages to limiting car usage first it can save money the cost of owning and operating a car can be significant including the purchase price of the car insurance gas and maintenance if you only use your car occasionally it may be more costeffective to use public transportation ridesharing or a carsharing service

second limiting car usage can help to reduce your environmental impact cars are a major source of air pollution and greenhouse gas emissions b

''

## model

In [7]:
import torch

model = AutoModelForCausalLM.from_pretrained('/root/autodl-tmp/ZhipuAI/glm-4-9b-chat', device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True)
model


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(151552, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-39): 40 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear(in_features=4096, out_features=4608, bias=True)
            (core_attention): SdpaAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_layer): Linear(in

In [8]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法


In [9]:
model.dtype


torch.bfloat16

## LoRA

In [10]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["query_key_value", "dense", "dense_h_to_4h", "dense_4h_to_h"],  # 现存问题只微调部分演示即可
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config


LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'dense_h_to_4h', 'dense_4h_to_h', 'dense', 'query_key_value'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [11]:
model = get_peft_model(model, config)
config


LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='/root/autodl-tmp/ZhipuAI/glm-4-9b-chat', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'dense_h_to_4h', 'dense_4h_to_h', 'dense', 'query_key_value'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [12]:
model.print_trainable_parameters()


trainable params: 21,176,320 || all params: 9,421,127,680 || trainable%: 0.22477479044207158


## train

In [13]:
# 定义训练参数
args = TrainingArguments(
    output_dir="./output/GLM4",  # 输出目录，用于保存模型和日志
    per_device_train_batch_size=1,  # 每个设备的训练批次大小
    gradient_accumulation_steps=8,  # 梯度累积步数，用于模拟更大的批次大小
    logging_steps=50,  # 每隔多少步记录一次日志
    num_train_epochs=2,  # 训练的总轮数
    save_steps=100,  # 每隔多少步保存一次模型
    learning_rate=1e-5,  # 学习率
    save_on_each_node=True,  # 是否在每个节点上保存模型
    gradient_checkpointing=True  # 是否启用梯度检查点（减少内存占用）
)


In [14]:
# 创建训练器实例
trainer = Trainer(
    model=model,  # 指定要训练的模型
    args=args,  # 传入训练参数
    train_dataset=tokenized_id,  # 提供训练数据集，这里假设已经进行了分词和编码处理
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),  # 使用适当的数据整理器，这里针对序列到序列任务进行填充
)

# 开始训练
trainer.train()


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
50,14008943.360000
100,396816.760000
150,8081.823100
200,220.480400
250,32608.627500
300,34.765900
350,139.260500
400,9456.833700
450,60.012500
500,421.507800


/root/miniconda3/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /root/autodl-tmp/ZhipuAI/glm-4-9b-chat - will assume that the vocabulary was not modified.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /root/autodl-tmp/ZhipuAI/glm-4-9b-chat - will assume that the vocabulary was not modified.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /root/autodl-tmp/ZhipuAI/glm-4-9b-chat - will assume that the vocabulary was not modified.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /root/autodl-tmp/ZhipuAI/glm-4-9b-chat - will assume that the vocabulary was not modified.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/peft/utils/save_and_load.p

TrainOutput(global_step=2250, training_loss=328369.2745628272, metrics={'train_runtime': 7603.5942, 'train_samples_per_second': 2.367, 'train_steps_per_second': 0.296, 'total_flos': 3.3814702738243584e+17, 'train_loss': 328369.2745628272, 'epoch': 2.0})

In [15]:
# 设置模型保存路径
peft_model_id = "./GLM4_lora"

# 保存训练好的模型到指定路径
trainer.model.save_pretrained(peft_model_id)

# 保存对应的分词器到指定路径
tokenizer.save_pretrained(peft_model_id)



/root/miniconda3/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /root/autodl-tmp/ZhipuAI/glm-4-9b-chat - will assume that the vocabulary was not modified.
  warnings.warn(


('./GLM4_lora/tokenizer_config.json',
 './GLM4_lora/special_tokens_map.json',
 './GLM4_lora/tokenizer.model',
 './GLM4_lora/added_tokens.json')

In [7]:
#合并
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = '/root/autodl-tmp/ZhipuAI/glm-4-9b-chat'
lora_path = './GLM4_lora'

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

## 推理

In [8]:
# 定义输入文本
input_text = "This is an example of a generated text."

# 构造分类任务的模板
inputs = tokenizer.apply_chat_template(
    [{"role": "user", "content": "Classify the text as either human-written or AI-generated."},
     {"role": "user", "content": f"Input: {input_text}"}],
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt",
    return_dict=True
).to('cuda')

# 推理参数
gen_kwargs = {"max_length": 100, "do_sample": True, "top_k": 1}

# 推理过程
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]  # 取出生成的部分
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

# 打印分类结果
print(f"Input: {input_text}")
print(f"Predicted Class: {prediction}")

Input: This is an example of a generated text.
Predicted Class: Output: AI-generated

The text provided is a simple statement that could be generated by an AI, as it lacks the complexity and context that human-written text often contains. However, without additional context or a more complex sample, it's challenging to be definitive. The classification is based on the simplicity and the nature of the statement.


## 测试

In [9]:

test_df

,instruction,input,output
6252,Classify the text as either human-written or A...,dear state senator\n\nim in favor of keeping t...,Human-written
4684,Classify the text as either human-written or A...,this new technology is actually valuable in th...,Human-written
1731,Classify the text as either human-written or A...,okay here is my attempt at writing an essay re...,AI-generated
4742,Classify the text as either human-written or A...,over the years the amount vf cars that have be...,Human-written
4521,Classify the text as either human-written or A...,however after reading the article driverless c...,AI-generated
...,...,...,...
3921,Classify the text as either human-written or A...,title the benefits of limiting car usage \n\nt...,AI-generated
6685,Classify the text as either human-written or A...,is the facial action coding system useful in c...,Human-written
3194,Classify the text as either human-written or A...,your name\nyour address\ncity state zip code\n...,AI-generated
1941,Classify the text as either human-written or A...,dear principle\n\nn think u should at least ha...,Human-written


In [19]:
from sklearn.metrics import accuracy_score
# 推理函数
def predict_class(input_text):
    # 构造分类任务的模板
    inputs = tokenizer.apply_chat_template(
        [{"role": "user", "content": "Classify the text as either human-written or AI-generated."},
         {"role": "user", "content": f"Input: {input_text}"}],
        add_generation_prompt=True,
        tokenize=True,
        return_tensors="pt",
        return_dict=True
    ).to('cuda')

    # 推理参数
    gen_kwargs = {"max_length": 2000, "do_sample": True, "top_k": 1}

    # 推理过程
    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs)
        outputs = outputs[:, inputs['input_ids'].shape[1]:]  # 取出生成的部分
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    return prediction

# 存储真实标签和预测标签
true_labels = []
predicted_labels = []

# 遍历测试集
# 遍历测试集的前 100 条数据
for idx, (_, row) in enumerate(test_df[:200].iterrows(), start=1):  # 使用 enumerate 并设置 start=1
    input_text = row['input']  # 输入文本
    true_label = row['output']  # 真实标签

    # 模型预测
    predicted_label = predict_class(input_text)

    # 将标签转换为统一的格式（例如，全部小写）
    true_label = true_label.lower().strip()
    predicted_label = predicted_label.lower().strip()

    # 添加到列表中
    true_labels.append(true_label)
    predicted_labels.append(predicted_label)

    # 打印当前样本的预测结果
    print(f"第 {idx} 条数据:")
    print(f"真实标签: {true_label}")
    print(f"预测标签: {predicted_label}")
    print("-" * 50)

# 计算准确率
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"测试集准确率: {accuracy * 100:.2f}%")

第 1 条数据:
真实标签: human-written
预测标签: human-written
--------------------------------------------------
第 2 条数据:
真实标签: human-written
预测标签: human-written
--------------------------------------------------
第 3 条数据:
真实标签: ai-generated
预测标签: ai-generated
--------------------------------------------------
第 4 条数据:
真实标签: human-written
预测标签: human-written
--------------------------------------------------
第 5 条数据:
真实标签: ai-generated
预测标签: ai-generated
--------------------------------------------------
第 6 条数据:
真实标签: human-written
预测标签: human-written
--------------------------------------------------
第 7 条数据:
真实标签: human-written
预测标签: human-written
--------------------------------------------------
第 8 条数据:
真实标签: human-written
预测标签: human-written
--------------------------------------------------
第 9 条数据:
真实标签: human-written
预测标签: human-written
--------------------------------------------------
第 10 条数据:
真实标签: human-written
预测标签: human-written
--------------------------------------------------
第 1